In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np
import tensorflow.keras.backend as K


In [3]:
# Load and preprocess MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize and reshape the data
x_train = x_train.reshape(-1, 28*28).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28*28).astype('float32') / 255.0

# Add Gaussian noise to the data (doubled noise factor)
noise_factor = 0.1  # Increased from 0.5 to 1.0
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

# Clip the noisy images to be between 0 and 1
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Create a simple MLP model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)
])

# Compile model with MSE loss
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['accuracy']
)

# Train the model with noisy data
history = model.fit(
    x_train_noisy, y_train,
    batch_size=128,
    epochs=10,
    validation_split=0.1,
    verbose=1
)

# Evaluate on test set with noisy data
test_loss, test_accuracy = model.evaluate(x_test_noisy, y_test, verbose=0)
print(f"\nTest accuracy (noisy): {test_accuracy:.4f}")
print(f"Test loss (MSE, noisy): {test_loss:.4f}")

# Make predictions on noisy test data and calculate accuracy manually to verify
y_pred = model.predict(x_test_noisy)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)
manual_accuracy = np.mean(y_pred_classes == y_test_classes)
print(f"Manual verification - Test accuracy (noisy): {manual_accuracy:.4f}")

# For comparison, evaluate on clean test data
clean_test_loss, clean_test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"\nTest accuracy (clean): {clean_test_accuracy:.4f}")
print(f"Test loss (MSE, clean): {clean_test_loss:.4f}")


Epoch 1/10
422/422 [==============================] - 8s 8ms/step - loss: 0.0296 - accuracy: 0.8708 - val_loss: 0.0144 - val_accuracy: 0.9550
Epoch 2/10
422/422 [==============================] - 3s 8ms/step - loss: 0.0133 - accuracy: 0.9510 - val_loss: 0.0102 - val_accuracy: 0.9685
Epoch 3/10
422/422 [==============================] - 3s 8ms/step - loss: 0.0097 - accuracy: 0.9663 - val_loss: 0.0082 - val_accuracy: 0.9733
Epoch 4/10
422/422 [==============================] - 3s 7ms/step - loss: 0.0077 - accuracy: 0.9742 - val_loss: 0.0074 - val_accuracy: 0.9752
Epoch 5/10
422/422 [==============================] - 3s 7ms/step - loss: 0.0063 - accuracy: 0.9800 - val_loss: 0.0067 - val_accuracy: 0.9767
Epoch 6/10
422/422 [==============================] - 4s 8ms/step - loss: 0.0053 - accuracy: 0.9844 - val_loss: 0.0061 - val_accuracy: 0.9782
Epoch 7/10
422/422 [==============================] - 3s 8ms/step - loss: 0.0046 - accuracy: 0.9876 - val_loss: 0.0056 - val_accuracy: 0.9785
Epoch 

In [4]:
# Create a custom loss function combining MSE and PCC
def custom_loss(y_true, y_pred):
    # MSE term
    mse = tf.reduce_mean(tf.square(y_pred - y_true))
    
    # PCC term
    y_true_centered = y_true - tf.reduce_mean(y_true) 
    y_pred_centered = y_pred - tf.reduce_mean(y_pred)
    
    cov = tf.reduce_sum(y_true_centered * y_pred_centered)
    std_y_true = tf.sqrt(tf.reduce_sum(tf.square(y_true_centered)))
    std_y_pred = tf.sqrt(tf.reduce_sum(tf.square(y_pred_centered)))
    
    pcc = cov / (std_y_true * std_y_pred + K.epsilon())
    
    # Calculate coefficient to match MSE range
    y_true_min = tf.reduce_min(y_true)
    y_true_max = tf.reduce_max(y_true)
    coef = tf.abs(y_true_max - y_true_min)  # Maximum possible MSE value
    
    # Combined loss with coefficient matching MSE range
    return mse + coef * (1.0 - pcc)

# Create the same MLP model
model_custom = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)
])

# Compile with custom loss
model_custom.compile(
    optimizer='adam',
    loss=custom_loss,
    metrics=['accuracy']
)

# Train the model with noisy data
history_custom = model_custom.fit(
    x_train_noisy, y_train,
    batch_size=128,
    epochs=10,
    validation_split=0.1,
    verbose=1
)

# Evaluate on test set with noisy data
test_loss_custom, test_accuracy_custom = model_custom.evaluate(x_test_noisy, y_test, verbose=0)
print(f"\nCustom Loss - Test accuracy (noisy): {test_accuracy_custom:.4f}")
print(f"Custom Loss - Test loss (noisy): {test_loss_custom:.4f}")

# Manual verification with noisy data
y_pred_custom = model_custom.predict(x_test_noisy)
y_pred_custom_classes = np.argmax(y_pred_custom, axis=1)
manual_accuracy_custom = np.mean(y_pred_custom_classes == y_test_classes)
print(f"Custom Loss - Manual verification accuracy (noisy): {manual_accuracy_custom:.4f}")

# For comparison, evaluate on clean test data
clean_test_loss_custom, clean_test_accuracy_custom = model_custom.evaluate(x_test, y_test, verbose=0)
print(f"\nCustom Loss - Test accuracy (clean): {clean_test_accuracy_custom:.4f}")
print(f"Custom Loss - Test loss (clean): {clean_test_loss_custom:.4f}")

# Compare results
print("\nComparison:")
print(f"Original MSE - Test accuracy (noisy): {test_accuracy:.4f}")
print(f"Custom Loss - Test accuracy (noisy): {test_accuracy_custom:.4f}")
print(f"Original MSE - Test accuracy (clean): {clean_test_accuracy:.4f}")
print(f"Custom Loss - Test accuracy (clean): {clean_test_accuracy_custom:.4f}")


Epoch 1/10
422/422 [==============================] - 5s 10ms/step - loss: 0.1775 - accuracy: 0.9011 - val_loss: 0.0830 - val_accuracy: 0.9632
Epoch 2/10
422/422 [==============================] - 4s 8ms/step - loss: 0.0786 - accuracy: 0.9586 - val_loss: 0.0596 - val_accuracy: 0.9702
Epoch 3/10
422/422 [==============================] - 4s 8ms/step - loss: 0.0562 - accuracy: 0.9720 - val_loss: 0.0495 - val_accuracy: 0.9740
Epoch 4/10
422/422 [==============================] - 4s 10ms/step - loss: 0.0442 - accuracy: 0.9788 - val_loss: 0.0437 - val_accuracy: 0.9758
Epoch 5/10
422/422 [==============================] - 4s 9ms/step - loss: 0.0361 - accuracy: 0.9834 - val_loss: 0.0400 - val_accuracy: 0.9783
Epoch 6/10
422/422 [==============================] - 4s 10ms/step - loss: 0.0301 - accuracy: 0.9869 - val_loss: 0.0370 - val_accuracy: 0.9792
Epoch 7/10
422/422 [==============================] - 4s 9ms/step - loss: 0.0253 - accuracy: 0.9901 - val_loss: 0.0350 - val_accuracy: 0.9792
Epo

In [5]:
# Load and preprocess MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize and reshape the data
x_train = x_train.reshape(-1, 28*28).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28*28).astype('float32') / 255.0

# Add Gaussian noise to the data (doubled noise factor)
noise_factor = 3  # Increased from 0.5 to 1.0
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

# Clip the noisy images to be between 0 and 1
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Create a simple MLP model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)
])

# Compile model with MSE loss
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['accuracy']
)

# Train the model with noisy data
history = model.fit(
    x_train_noisy, y_train,
    batch_size=128,
    epochs=10,
    validation_split=0.1,
    verbose=1
)

# Evaluate on test set with noisy data
test_loss, test_accuracy = model.evaluate(x_test_noisy, y_test, verbose=0)
print(f"\nTest accuracy (noisy): {test_accuracy:.4f}")
print(f"Test loss (MSE, noisy): {test_loss:.4f}")

# Make predictions on noisy test data and calculate accuracy manually to verify
y_pred = model.predict(x_test_noisy)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)
manual_accuracy = np.mean(y_pred_classes == y_test_classes)
print(f"Manual verification - Test accuracy (noisy): {manual_accuracy:.4f}")

# For comparison, evaluate on clean test data
clean_test_loss, clean_test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"\nTest accuracy (clean): {clean_test_accuracy:.4f}")
print(f"Test loss (MSE, clean): {clean_test_loss:.4f}")


Epoch 1/10
422/422 [==============================] - 4s 9ms/step - loss: 0.0949 - accuracy: 0.1101 - val_loss: 0.0902 - val_accuracy: 0.1192
Epoch 2/10
422/422 [==============================] - 3s 8ms/step - loss: 0.0897 - accuracy: 0.1337 - val_loss: 0.0895 - val_accuracy: 0.1263
Epoch 3/10
422/422 [==============================] - 3s 8ms/step - loss: 0.0887 - accuracy: 0.1693 - val_loss: 0.0883 - val_accuracy: 0.1828
Epoch 4/10
422/422 [==============================] - 3s 7ms/step - loss: 0.0873 - accuracy: 0.2076 - val_loss: 0.0870 - val_accuracy: 0.2157
Epoch 5/10
422/422 [==============================] - 3s 7ms/step - loss: 0.0861 - accuracy: 0.2270 - val_loss: 0.0861 - val_accuracy: 0.2287
Epoch 6/10
422/422 [==============================] - 3s 7ms/step - loss: 0.0854 - accuracy: 0.2393 - val_loss: 0.0857 - val_accuracy: 0.2343
Epoch 7/10
422/422 [==============================] - 3s 7ms/step - loss: 0.0848 - accuracy: 0.2533 - val_loss: 0.0851 - val_accuracy: 0.2502
Epoch 

In [6]:
# Create a custom loss function combining MSE and PCC
def custom_loss(y_true, y_pred):
    # MSE term
    mse = tf.reduce_mean(tf.square(y_pred - y_true))
    
    # PCC term
    y_true_centered = y_true - tf.reduce_mean(y_true) 
    y_pred_centered = y_pred - tf.reduce_mean(y_pred)
    
    cov = tf.reduce_sum(y_true_centered * y_pred_centered)
    std_y_true = tf.sqrt(tf.reduce_sum(tf.square(y_true_centered)))
    std_y_pred = tf.sqrt(tf.reduce_sum(tf.square(y_pred_centered)))
    
    pcc = cov / (std_y_true * std_y_pred + K.epsilon())
    
    # Calculate coefficient to match MSE range
    y_true_min = tf.reduce_min(y_true)
    y_true_max = tf.reduce_max(y_true)
    coef = tf.abs(y_true_max - y_true_min)  # Maximum possible MSE value
    
    # Combined loss with coefficient matching MSE range
    return mse + coef * (1.0 - pcc)

# Create the same MLP model
model_custom = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)
])

# Compile with custom loss
model_custom.compile(
    optimizer='adam',
    loss=custom_loss,
    metrics=['accuracy']
)

# Train the model with noisy data
history_custom = model_custom.fit(
    x_train_noisy, y_train,
    batch_size=128,
    epochs=10,
    validation_split=0.1,
    verbose=1
)

# Evaluate on test set with noisy data
test_loss_custom, test_accuracy_custom = model_custom.evaluate(x_test_noisy, y_test, verbose=0)
print(f"\nCustom Loss - Test accuracy (noisy): {test_accuracy_custom:.4f}")
print(f"Custom Loss - Test loss (noisy): {test_loss_custom:.4f}")

# Manual verification with noisy data
y_pred_custom = model_custom.predict(x_test_noisy)
y_pred_custom_classes = np.argmax(y_pred_custom, axis=1)
manual_accuracy_custom = np.mean(y_pred_custom_classes == y_test_classes)
print(f"Custom Loss - Manual verification accuracy (noisy): {manual_accuracy_custom:.4f}")

# For comparison, evaluate on clean test data
clean_test_loss_custom, clean_test_accuracy_custom = model_custom.evaluate(x_test, y_test, verbose=0)
print(f"\nCustom Loss - Test accuracy (clean): {clean_test_accuracy_custom:.4f}")
print(f"Custom Loss - Test loss (clean): {clean_test_loss_custom:.4f}")

# Compare results
print("\nComparison:")
print(f"Original MSE - Test accuracy (noisy): {test_accuracy:.4f}")
print(f"Custom Loss - Test accuracy (noisy): {test_accuracy_custom:.4f}")
print(f"Original MSE - Test accuracy (clean): {clean_test_accuracy:.4f}")
print(f"Custom Loss - Test accuracy (clean): {clean_test_accuracy_custom:.4f}")


Epoch 1/10
422/422 [==============================] - 5s 10ms/step - loss: 0.9966 - accuracy: 0.1639 - val_loss: 0.8944 - val_accuracy: 0.2215
Epoch 2/10
422/422 [==============================] - 4s 10ms/step - loss: 0.8719 - accuracy: 0.2389 - val_loss: 0.8904 - val_accuracy: 0.2243
Epoch 3/10
422/422 [==============================] - 4s 10ms/step - loss: 0.8442 - accuracy: 0.2546 - val_loss: 0.8562 - val_accuracy: 0.2495
Epoch 4/10
422/422 [==============================] - 4s 10ms/step - loss: 0.8269 - accuracy: 0.2658 - val_loss: 0.8444 - val_accuracy: 0.2632
Epoch 5/10
422/422 [==============================] - 4s 9ms/step - loss: 0.8089 - accuracy: 0.2779 - val_loss: 0.8419 - val_accuracy: 0.2600
Epoch 6/10
422/422 [==============================] - 4s 10ms/step - loss: 0.7968 - accuracy: 0.2882 - val_loss: 0.8394 - val_accuracy: 0.2573
Epoch 7/10
422/422 [==============================] - 4s 10ms/step - loss: 0.7784 - accuracy: 0.3022 - val_loss: 0.8248 - val_accuracy: 0.2720


In [28]:
# Load and preprocess MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize and reshape the data
x_train = x_train.reshape(-1, 28*28).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28*28).astype('float32') / 255.0

# Create imbalanced dataset with variable ratios
# Define imbalance ratios for each digit (0-9)
imbalance_ratios = {
    0: 1.0,    # Keep all samples
    1: 0.8,    # Keep 80% of samples
    2: 0.6,    # Keep 60% of samples
    3: 0.4,    # Keep 40% of samples
    4: 0.3,    # Keep 30% of samples
    5: 0.25,   # Keep 25% of samples
    6: 0.2,    # Keep 20% of samples
    7: 0.15,   # Keep 15% of samples
    8: 0.1,    # Keep 10% of samples
    9: 0.05    # Keep 5% of samples
}

# Create masks for imbalanced sampling
train_indices = []
for digit in range(10):
    digit_indices = np.where(y_train == digit)[0]
    num_samples = int(len(digit_indices) * imbalance_ratios[digit])
    selected_indices = np.random.choice(digit_indices, num_samples, replace=False)
    train_indices.extend(selected_indices)

train_indices = np.array(train_indices)
np.random.shuffle(train_indices)

# Select the imbalanced subset
x_train = x_train[train_indices]
y_train = y_train[train_indices]

# Add Gaussian noise to the data
noise_factor = 1.0
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape)

# Clip the noisy images to be between 0 and 1
x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Print class distribution
print("Class distribution in training set:")
for i in range(10):
    count = np.sum(np.argmax(y_train, axis=1) == i)
    print(f"Digit {i}: {count} samples")

# Create a simple MLP model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)
])

# Compile model with MSE loss
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['accuracy']
)

# Train the model with noisy data
history = model.fit(
    x_train_noisy, y_train,
    batch_size=256,
    epochs=10,
    validation_split=0.1,
    verbose=1
)

# Evaluate on test set with noisy data
test_loss, test_accuracy = model.evaluate(x_test_noisy, y_test, verbose=0)
print(f"\nTest accuracy (noisy): {test_accuracy:.4f}")
print(f"Test loss (MSE, noisy): {test_loss:.4f}")

# Make predictions and calculate per-class accuracy
y_pred = model.predict(x_test_noisy)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

print("\nPer-class accuracy:")
for i in range(10):
    mask = (y_test_classes == i)
    class_acc = np.mean(y_pred_classes[mask] == y_test_classes[mask])
    print(f"Digit {i}: {class_acc:.4f}")

# For comparison, evaluate on clean test data
clean_test_loss, clean_test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"\nTest accuracy (clean): {clean_test_accuracy:.4f}")
print(f"Test loss (MSE, clean): {clean_test_loss:.4f}")


Class distribution in training set:
Digit 0: 5923 samples
Digit 1: 5393 samples
Digit 2: 3574 samples
Digit 3: 2452 samples
Digit 4: 1752 samples
Digit 5: 1355 samples
Digit 6: 1183 samples
Digit 7: 939 samples
Digit 8: 585 samples
Digit 9: 297 samples
Epoch 1/10
83/83 [==============================] - 1s 8ms/step - loss: 0.0969 - accuracy: 0.3525 - val_loss: 0.0711 - val_accuracy: 0.4817
Epoch 2/10
83/83 [==============================] - 1s 8ms/step - loss: 0.0644 - accuracy: 0.5267 - val_loss: 0.0587 - val_accuracy: 0.5759
Epoch 3/10
83/83 [==============================] - 1s 9ms/step - loss: 0.0551 - accuracy: 0.6121 - val_loss: 0.0528 - val_accuracy: 0.6394
Epoch 4/10
83/83 [==============================] - 1s 9ms/step - loss: 0.0497 - accuracy: 0.6682 - val_loss: 0.0492 - val_accuracy: 0.6675
Epoch 5/10
83/83 [==============================] - 1s 10ms/step - loss: 0.0462 - accuracy: 0.6956 - val_loss: 0.0479 - val_accuracy: 0.6718
Epoch 6/10
83/83 [============================

In [29]:
# Create a custom loss function combining MSE and PCC
def custom_loss(y_true, y_pred):
    # MSE term
    mse = tf.reduce_mean(tf.square(y_pred - y_true))
    
    # PCC term
    y_true_centered = y_true - tf.reduce_mean(y_true) 
    y_pred_centered = y_pred - tf.reduce_mean(y_pred)
    
    cov = tf.reduce_sum(y_true_centered * y_pred_centered)
    std_y_true = tf.sqrt(tf.reduce_sum(tf.square(y_true_centered)))
    std_y_pred = tf.sqrt(tf.reduce_sum(tf.square(y_pred_centered)))
    
    pcc = cov / (std_y_true * std_y_pred + K.epsilon())
    
    # Calculate coefficient to match MSE range
    y_true_min = tf.reduce_min(y_true)
    y_true_max = tf.reduce_max(y_true)
    coef = tf.abs(y_true_max - y_true_min)  # Maximum possible MSE value
    
    # Combined loss with coefficient matching MSE range
    return mse + coef * (1.0 - pcc)

# Create the same MLP model
model_custom = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)
])

# Compile with custom loss
model_custom.compile(
    optimizer='adam',
    loss=custom_loss,
    metrics=['accuracy']
)

# Train the model with noisy data
history_custom = model_custom.fit(
    x_train_noisy, y_train,
    batch_size=256,
    epochs=10,
    validation_split=0.1,
    verbose=1
)

# Evaluate on test set with noisy data
test_loss_custom, test_accuracy_custom = model_custom.evaluate(x_test_noisy, y_test, verbose=0)
print(f"\nCustom Loss - Test accuracy (noisy): {test_accuracy_custom:.4f}")
print(f"Custom Loss - Test loss (noisy): {test_loss_custom:.4f}")

# Make predictions and calculate per-class accuracy
y_pred_custom = model_custom.predict(x_test_noisy)
y_pred_custom_classes = np.argmax(y_pred_custom, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

print("\nCustom Loss - Per-class accuracy:")
for i in range(10):
    mask = (y_test_classes == i)
    class_acc = np.mean(y_pred_custom_classes[mask] == y_test_classes[mask])
    print(f"Digit {i}: {class_acc:.4f}")

# For comparison, evaluate on clean test data
clean_test_loss_custom, clean_test_accuracy_custom = model_custom.evaluate(x_test, y_test, verbose=0)
print(f"\nCustom Loss - Test accuracy (clean): {clean_test_accuracy_custom:.4f}")
print(f"Custom Loss - Test loss (clean): {clean_test_loss_custom:.4f}")

# Compare results
print("\nComparison:")
print(f"Original MSE - Test accuracy (noisy): {test_accuracy:.4f}")
print(f"Custom Loss - Test accuracy (noisy): {test_accuracy_custom:.4f}")
print(f"Original MSE - Test accuracy (clean): {clean_test_accuracy:.4f}")
print(f"Custom Loss - Test accuracy (clean): {clean_test_accuracy_custom:.4f}")


Epoch 1/10
83/83 [==============================] - 1s 7ms/step - loss: 0.6363 - accuracy: 0.4705 - val_loss: 0.4770 - val_accuracy: 0.5882
Epoch 2/10
83/83 [==============================] - 1s 10ms/step - loss: 0.4196 - accuracy: 0.6368 - val_loss: 0.3984 - val_accuracy: 0.6505
Epoch 3/10
83/83 [==============================] - 1s 7ms/step - loss: 0.3523 - accuracy: 0.6954 - val_loss: 0.3557 - val_accuracy: 0.6888
Epoch 4/10
83/83 [==============================] - 0s 6ms/step - loss: 0.3096 - accuracy: 0.7321 - val_loss: 0.3302 - val_accuracy: 0.7008
Epoch 5/10
83/83 [==============================] - 1s 9ms/step - loss: 0.2803 - accuracy: 0.7587 - val_loss: 0.3205 - val_accuracy: 0.7140
Epoch 6/10
83/83 [==============================] - 1s 12ms/step - loss: 0.2610 - accuracy: 0.7766 - val_loss: 0.3124 - val_accuracy: 0.7208
Epoch 7/10
83/83 [==============================] - 1s 8ms/step - loss: 0.2437 - accuracy: 0.7912 - val_loss: 0.3025 - val_accuracy: 0.7191
Epoch 8/10
83/83 [